In [1]:
print("Hi")

Hi


In [1]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
#api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

api_key = 'c4f6971b-50d6-4c70-ac25-6b02638c31f1'

# configure client
pc = Pinecone(api_key=api_key)

In [2]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [3]:
index_name = 'semantic'

In [4]:
import time

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of minilm
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 15}, 'ns1': {'vector_count': 1}},
 'total_vector_count': 16}

In [5]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# model

In [6]:
model1 = SentenceTransformer('all-mpnet-base-v2')

In [41]:
from tqdm.auto import tqdm
import json

with open("conversations.json", "r") as f:
    data = json.load(f)

#print(data)

for key, value in data.items():
    # Upsert the embeddings into Pinecone index
    index.upsert(vectors=[{'id':str({key:value}), 'values': model1.encode(key).tolist()}])
    



In [29]:
import json
with open("conversations.json", "r") as f:
    data = json.load(f)

print(data)

{'PM of India': 'The current Prime Minister of India is Narendra Modi. He has been in office since May 2014, after his party, the Bharatiya Janata Party (BJP), won the general elections. Narendra Modi was re-elected for a second term in May 2019.', 'CM of AP': 'The current Chief Minister of Andhra Pradesh is Y. S. Jaganmohan Reddy. He assumed office on 30 May 2019 after his party, YSR Congress Party, won the state assembly elections.'}


In [37]:
for key, value in data.items():
    # Upsert the embeddings into Pinecone index
    index.upsert(vectors=[{'id': key, 'values': model1.encode(value).tolist()}])

SyntaxError: ':' expected after dictionary key (4208656704.py, line 3)

In [7]:
# exmaple=["What is the capital of France?",
# "Who invented the lightbulb?",
# "How do plants convert sunlight into energy?",
# "What causes earthquakes?",
# "What are the symptoms of COVID-19?",
# "How does the internet work?",
# "What is the meaning of life?",
# "Why do we dream?",
# "What is the difference between weather and climate?",
# "How do vaccines work?"]
# len(exmaple)
exmaple=["How many shipments are delivered to germany in sep 2022"]

In [8]:
for i,j in enumerate(exmaple):
    # Upsert the embeddings into Pinecone index
    index.upsert(vectors=[{'id': str(j), 'values': model1.encode(exmaple[i]).tolist()}])
# # Encode the text
# embeddings = model.encode("How are you?")

# # Upsert the embeddings into Pinecone index
# index.upsert(vectors=[{'id': 'how_are_you', 'values': embeddings.tolist()}])

In [21]:
query = "How many shipments are stolen from germany in sep 2022"
xq=model1.encode([query]).tolist()
#result = index.query(vectors=xq, top_k=2)


results = index.query(
    vector=xq,
    top_k=1,
    include_metadata=True
    #include_values=True
)
print(results)

for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['id']}")
    print("Answer is: ",type(result['id']))
    

{'matches': [{'id': 'How many shipments are delivered to germany in sep 2022',
              'score': 0.799638808,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}
0.8: How many shipments are delivered to germany in sep 2022
Answer is:  <class 'str'>


In [70]:
import ast
for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['id']}")
    # Convert string to dictionary
    converted_dict = ast.literal_eval(result['id'])
    print("Answer is: ",list(converted_dict.values()))

0.89: {'PM of India': 'The current Prime Minister of India is Narendra Modi. He has been in office since May 2014, after his party, the Bharatiya Janata Party (BJP), won the general elections. Narendra Modi was re-elected for a second term in May 2019.'}
Answer is:  ['The current Prime Minister of India is Narendra Modi. He has been in office since May 2014, after his party, the Bharatiya Janata Party (BJP), won the general elections. Narendra Modi was re-elected for a second term in May 2019.']


In [75]:
questions_and_answers = {
    "What is the capital of France?": "The capital of France is Paris.",
    "Who wrote 'Romeo and Juliet'?": "William Shakespeare wrote 'Romeo and Juliet'.",
    "What is the boiling point of water in Celsius?": "The boiling point of water in Celsius is 100 degrees.",
    "What is the largest planet in our solar system?": "The largest planet in our solar system is Jupiter.",
    "Who painted the Mona Lisa?": "Leonardo da Vinci painted the Mona Lisa.",
    #"What is the formula for the area of a circle?": "The formula for the area of a circle is πr^2, where r is the radius of the circle.",
    "What year did the Titanic sink?": "The Titanic sank in 1912.",
    "Who is the current President of the United States?": "As of my last update in January 2022, the President of the United States is Joe Biden."
}

In [76]:
for key, value in questions_and_answers.items():
    # Upsert the embeddings into Pinecone index
    index.upsert(vectors=[{'id':str({key:value}), 'values': model1.encode(key).tolist()}])

In [8]:
import ast

In [13]:
query = "Titanic disapperaed in which year?"
xq=model1.encode([query]).tolist()
#result = index.query(vectors=xq, top_k=2)


results = index.query(
    vector=xq,
    top_k=1,
    include_metadata=True
    #include_values=True
)
#print(results)

for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['id']}")
    #print("Answer is: ",type(result['id']))
    converted_dict = ast.literal_eval(result['id'])
    print("Question is: ",list(converted_dict.keys()))
    print("Answer is: ",list(converted_dict.values()))

0.87: {'What year did the Titanic sink?': 'The Titanic sank in 1912.'}
Question is:  ['What year did the Titanic sink?']
Answer is:  ['The Titanic sank in 1912.']


In [14]:
ques_ans={
    "What are the environmental implications of deforestation in the Amazon rainforest?":
    "Deforestation in the Amazon rainforest leads to the loss of biodiversity, disruption of ecosystems, release of carbon dioxide into the atmosphere, and exacerbation of climate change.",
    
    "How does the human brain process and store long-term memories?":
    "Long-term memories are formed through a process involving the consolidation of information in various regions of the brain, such as the hippocampus and cortex, where synaptic connections are strengthened, allowing for the storage of memories over extended periods.",
    
    "Explain the role of neurotransmitters in mood regulation and mental health.":
    "Neurotransmitters such as serotonin, dopamine, and norepinephrine play crucial roles in regulating mood, emotions, and cognitive functions. Imbalances in these neurotransmitters are associated with various mental health disorders like depression, anxiety, and schizophrenia.",
    
    "What are the potential economic impacts of a global pandemic like COVID-19?":
    "Global pandemics like COVID-19 can lead to economic recessions, job losses, disruptions in supply chains, decreased consumer spending, and increased government spending on healthcare and relief efforts, causing significant economic downturns worldwide.",
    
    "Describe the process of gene expression and its significance in cellular functions.":
    "Gene expression involves the transcription of DNA into mRNA and translation of mRNA into proteins, which are essential for carrying out various cellular functions such as metabolism, growth, and repair. Regulation of gene expression is crucial for maintaining cellular homeostasis and responding to environmental cues.",
    
    "How do renewable energy sources like solar and wind power contribute to mitigating climate change?":
    "Renewable energy sources like solar and wind power generate electricity without producing greenhouse gas emissions, thereby reducing reliance on fossil fuels and mitigating climate change by decreasing carbon dioxide emissions and other pollutants.",
    
    "Discuss the impact of globalization on cultural diversity and identity.":
    "Globalization has both positive and negative effects on cultural diversity and identity. While it promotes cultural exchange and integration, it can also lead to the homogenization of cultures, loss of traditional practices, and erosion of indigenous languages and customs.",
    
    "What are the ethical considerations surrounding the use of artificial intelligence in decision-making processes?":
    "Ethical considerations surrounding the use of artificial intelligence include issues related to bias, transparency, accountability, privacy, and the potential for AI systems to reinforce existing inequalities or infringe upon human rights.",
    
    "Explain the relationship between socioeconomic status and health outcomes.":
    "Socioeconomic status, including factors such as income, education, and access to healthcare, significantly influences health outcomes. Individuals with higher socioeconomic status generally have better access to resources and healthcare services, leading to improved health outcomes compared to those with lower socioeconomic status.",
    
    "What are the mechanisms underlying antibiotic resistance in bacteria, and how can it be addressed?":
    "Antibiotic resistance in bacteria occurs through mechanisms such as genetic mutations, horizontal gene transfer, and the overuse or misuse of antibiotics. Addressing antibiotic resistance requires strategies such as prudent antibiotic use, development of new antibiotics, and investment in alternative treatments like phage therapy."
}


In [15]:
for key, value in ques_ans.items():
    # Upsert the embeddings into Pinecone index
    index.upsert(vectors=[{'id':str({key:value}), 'values': model1.encode(key).tolist()}])

In [23]:
query = "effects of COVID-19"
xq=model1.encode([query]).tolist()
#result = index.query(vectors=xq, top_k=2)


results = index.query(
    vector=xq,
    top_k=1,
    include_metadata=True
    #include_values=True
)
#print(results)

for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['id']}")
    #print("Answer is: ",type(result['id']))
    converted_dict = ast.literal_eval(result['id'])
    print("Question is: ",list(converted_dict.keys()))
    print("Answer is: ",list(converted_dict.values()))

0.66: {'What are the potential economic impacts of a global pandemic like COVID-19?': 'Global pandemics like COVID-19 can lead to economic recessions, job losses, disruptions in supply chains, decreased consumer spending, and increased government spending on healthcare and relief efforts, causing significant economic downturns worldwide.'}
Question is:  ['What are the potential economic impacts of a global pandemic like COVID-19?']
Answer is:  ['Global pandemics like COVID-19 can lead to economic recessions, job losses, disruptions in supply chains, decreased consumer spending, and increased government spending on healthcare and relief efforts, causing significant economic downturns worldwide.']


In [16]:
ind="Earth, the third celestial body from the Sun in our solar system, stands as a testament to the wonders of the universe. Spanning approximately 12,742 kilometers in diameter and boasting a mass of about 5.97 × 10^24 kilograms, Earth is a dynamic and diverse world teeming with life. Its layered structure, comprising a solid inner core, liquid outer core, semi-solid mantle, and thin crust, facilitates a plethora of geological processes that have shaped its surface over billions of years. Enveloping the planet is its atmosphere, primarily composed of nitrogen (78%) and oxygen (21%), which sustains life and moderates temperatures, creating the perfect conditions for a wide array of ecosystems to flourish. Earth's vast oceans, covering more than 70% of its surface, play a pivotal role in regulating climate, supporting biodiversity, and shaping weather patterns, serving as vital sources of sustenance and livelihood for countless species, including humans. The planet's magnetic field, generated by its churning outer core, shields it from harmful solar radiation and cosmic particles, safeguarding life as we know it. Rotating on its axis, Earth experiences the rhythm of day and night, while its orbit around the Sun dictates the changing seasons, marking the passage of time. Human civilization has thrived on Earth, harnessing its abundant resources for food, shelter, energy, and innovation, shaping landscapes and societies across continents and millennia. However, human activities have also left indelible marks on the planet, causing widespread environmental degradation, pollution, and climate change, threatening the delicate balance of life. Despite these challenges, Earth remains a beacon of resilience, beauty, and interconnectedness, reminding us of the importance of conservation, sustainability, and stewardship for the future of all life on this precious blue planet."
len(ind)

1892

In [3]:
pharma_examples={"How do SSRIs work in the treatment of depression?": "Selective serotonin reuptake inhibitors (SSRIs) work by blocking the reabsorption (reuptake) of serotonin in the brain, which helps to increase the levels of serotonin available to nerve cells. This is thought to improve mood and alleviate symptoms of depression.",

"What are the different classes of antibiotics?": "Antibiotics can be classified into several categories based on their mechanism of action, chemical structure, and spectrum of activity. Some common classes include penicillins, cephalosporins, macrolides, tetracyclines, fluoroquinolones, aminoglycosides, sulfonamides, and carbapenems.",

"What is the mechanism of action of statins?": "Statins work by inhibiting the enzyme HMG-CoA reductase, which is involved in the synthesis of cholesterol in the liver. By blocking this enzyme, statins reduce the production of cholesterol, leading to lower levels of LDL cholesterol in the blood.",

"What are the potential side effects of NSAIDs?": "Nonsteroidal anti-inflammatory drugs (NSAIDs) can cause a range of side effects, including gastrointestinal issues such as stomach ulcers and bleeding, kidney problems, cardiovascular risks like heart attack and stroke, and allergic reactions in some individuals.",

"What are the different types of insulin and their onset of action?": "Insulin types include rapid-acting (onset within 10-30 minutes), short-acting (onset within 30 minutes to an hour), intermediate-acting (onset within 1-2 hours), and long-acting (onset within several hours, lasting up to 24 hours). Each type has a different duration of action and is used to manage blood sugar levels in people with diabetes.",

"How do proton pump inhibitors (PPIs) work in the treatment of gastric ulcers?": "Proton pump inhibitors (PPIs) work by irreversibly blocking the proton pump in the stomach's parietal cells, which reduces the production of gastric acid. This helps to heal gastric ulcers by reducing the acidity in the stomach and allowing the ulcer to heal.",

"What is the mechanism of action of ACE inhibitors in treating hypertension?": "Angiotensin-converting enzyme (ACE) inhibitors work by inhibiting the conversion of angiotensin I to angiotensin II, a potent vasoconstrictor. By blocking this conversion, ACE inhibitors reduce blood pressure by relaxing blood vessels and decreasing the workload of the heart.",

"How do beta-blockers work in the management of cardiac arrhythmias?": "Beta-blockers work by blocking the action of adrenaline and noradrenaline on beta-adrenergic receptors in the heart. This slows down the heart rate, reduces the force of contraction, and decreases the electrical conductivity in the heart, helping to control cardiac arrhythmias.",

"What are the different types of anticoagulants and their mechanisms of action?": "Anticoagulants can be classified into direct thrombin inhibitors, such as dabigatran, and factor Xa inhibitors, such as rivaroxaban and apixaban. These medications interfere with the blood clotting process by targeting specific factors involved in coagulation, thereby preventing the formation of blood clots.",

"How do diuretics work in the treatment of hypertension?": "Diuretics work by increasing the excretion of sodium and water from the body through the kidneys. This helps to reduce the volume of blood in the bloodstream, leading to a decrease in blood pressure. Diuretics are commonly used as first-line agents in the management of hypertension.",

"What is the mechanism of action of benzodiazepines in treating anxiety disorders?": "Benzodiazepines work by enhancing the activity of gamma-aminobutyric acid (GABA), a neurotransmitter that inhibits the activity of neurons in the brain. By increasing GABAergic activity, benzodiazepines produce calming and sedative effects, which can help alleviate symptoms of anxiety disorders.",

"How do antihistamines work in the treatment of allergic reactions?": "Antihistamines work by blocking the action of histamine, a chemical released by the immune system in response to allergens. By inhibiting histamine receptors, antihistamines help to reduce symptoms of allergic reactions such as itching, sneezing, and nasal congestion.",

"What are the mechanisms of action of opioid analgesics?": "Opioid analgesics work by binding to opioid receptors in the brain, spinal cord, and other parts of the body. This binding activates the body's natural pain-relief system, leading to a reduction in the perception of pain. Opioids can also produce other effects such as sedation, respiratory depression, and euphoria.",

"How do antipsychotic medications work in the treatment of schizophrenia?": "Antipsychotic medications work by blocking dopamine receptors in the brain, particularly in the mesolimbic pathway. By reducing dopamine activity in this pathway, antipsychotics help to alleviate symptoms of psychosis, such as hallucinations and delusions, in people with schizophrenia.",

"What are the different classes of antidepressant medications?": "Antidepressant medications can be classified into several categories, including selective serotonin reuptake inhibitors (SSRIs), serotonin-norepinephrine reuptake inhibitors (SNRIs), tricyclic antidepressants (TCAs), and monoamine oxidase inhibitors (MAOIs). Each class has a different mechanism of action and side effect profile.",

"How do calcium channel blockers work in the treatment of hypertension?": "Calcium channel blockers work by blocking the influx of calcium ions into smooth muscle cells in blood vessels and the heart. This causes relaxation of the blood vessels and decreases the force of contraction of the heart, leading to a reduction in blood pressure.",

"What is the mechanism of action of corticosteroids in the treatment of inflammatory conditions?": "Corticosteroids work by binding to glucocorticoid receptors in cells throughout the body, leading to the activation or repression of various genes involved in the immune response and inflammation. This helps to suppress inflammation and immune responses in conditions such as asthma, rheumatoid arthritis, and allergic reactions.",

"How do anticonvulsant medications work in the management of epilepsy?": "Anticonvulsant medications work by stabilizing the electrical activity of neurons in the brain, making them less likely to become hyperexcitable and generate abnormal electrical impulses that cause seizures. Different anticonvulsants may act on various neurotransmitter systems or ion channels involved in neuronal excitability.",

"What are the mechanisms of action of immunosuppressant drugs?": "Immunosuppressant drugs work by suppressing the activity of the immune system, primarily by inhibiting the proliferation and function of immune cells such as T cells and B cells. This helps to prevent the rejection of transplanted organs and the progression of autoimmune diseases by reducing inflammation and immune-mediated damage."}

In [18]:
new_data={"Describe about Earth":"Earth, the third celestial body from the Sun in our solar system, stands as a testament to the wonders of the universe. Spanning approximately 12,742 kilometers in diameter and boasting a mass of about 5.97 × 10^24 kilograms, Earth is a dynamic and diverse world teeming with life. Its layered structure, comprising a solid inner core, liquid outer core, semi-solid mantle, and thin crust, facilitates a plethora of geological processes that have shaped its surface over billions of years. Enveloping the planet is its atmosphere, primarily composed of nitrogen (78%) and oxygen (21%), which sustains life and moderates temperatures, creating the perfect conditions for a wide array of ecosystems to flourish. Earth's vast oceans, covering more than 70% of its surface, play a pivotal role in regulating climate, supporting biodiversity, and shaping weather patterns, serving as vital sources of sustenance and livelihood for countless species, including humans. The planet's magnetic field, generated by its churning outer core, shields it from harmful solar radiation and cosmic particles, safeguarding life as we know it. Rotating on its axis, Earth experiences the rhythm of day and night, while its orbit around the Sun dictates the changing seasons, marking the passage of time. Human civilization has thrived on Earth, harnessing its abundant resources for food, shelter, energy, and innovation, shaping landscapes and societies across continents and millennia. However, human activities have also left indelible marks on the planet, causing widespread environmental degradation, pollution, and climate change, threatening the delicate balance of life. Despite these challenges, Earth remains a beacon of resilience, beauty, and interconnectedness, reminding us of the importance of conservation, sustainability, and stewardship for the future of all life on this precious blue planet."}

In [20]:
for key, value in new_data.items():
    # Upsert the embeddings into Pinecone index
    index.upsert(vectors=[{'id':key, 'values': model1.encode(key).tolist(), 'metadata': {'Question': key, 'Answer': value}}])

In [12]:
import ast

In [28]:
query = "Tell me about Earth"
xq=model1.encode([query]).tolist()
#result = index.query(vectors=xq, top_k=2)


results = index.query(
    vector=xq,
    top_k=1,
    include_metadata=True
    #include_values=True
)
print(results)

for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['id']}")
    print("Metadata is: ",result['metadata'])
    print("Metadata Question is: ",list(result['metadata'].keys()))
    print("Metadata Answer is: ",list(result['metadata'].values())[1])
    # converted_dict = ast.literal_eval(result['id'])
    # print("Question is: ",list(converted_dict.keys()))
    # print("Answer is: ",list(converted_dict.values()))

{'matches': [{'id': 'Describe about Earth',
              'metadata': {'Answer': 'Earth, the third celestial body from the '
                                     'Sun in our solar system, stands as a '
                                     'testament to the wonders of the '
                                     'universe. Spanning approximately 12,742 '
                                     'kilometers in diameter and boasting a '
                                     'mass of about 5.97 × 10^24 kilograms, '
                                     'Earth is a dynamic and diverse world '
                                     'teeming with life. Its layered '
                                     'structure, comprising a solid inner '
                                     'core, liquid outer core, semi-solid '
                                     'mantle, and thin crust, facilitates a '
                                     'plethora of geological processes that '
                                  

In [4]:
import hashlib
def hash_password(password):
    # Hash the password using SHA-256 algorithm
    return hashlib.sha256(password.encode()).hexdigest()
    

hash_password("master123")

'e7bc2f973afb8dfaf00fadfb19596741108be08ab4a107c6a799c429b684c64a'

In [1]:
from datetime import date

# date object of today's date
today = date.today()
print("Today's date:", today)

# Create a date object from the string
target_date = date(2024, 6, 4)

# Check if today's date matches the target date
if today > target_date:
    print("Today is the target date")
else:
    print("Today is not the target date")


Today's date: 2024-06-05
Today is the target date


In [5]:
from pinecone import Pinecone
api_key = 'c4f6971b-50d6-4c70-ac25-6b02638c31f1'
# configure client
pc = Pinecone(api_key=api_key)

index_name = "semantic"

# connect to index
index = pc.Index(index_name)

print(index.describe_index_stats())
print(list(index.describe_index_stats()["namespaces"].keys()))

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}, 'ns1': {'vector_count': 1}},
 'total_vector_count': 5}
['', 'ns1']


In [9]:
#Get all vectors from the index with metadata
vectors = index.fetch(ids=["What is YOLO in AI","dog"])
print(vectors)

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'dog': {'id': 'dog',
                     'metadata': {'Answer': 'SELECT DISTINCT CUST_GRP1 AS '
                                            '`Chief Ministers` FROM orders',
                                  'Question': 'dog'},
                     'values': [0.0215680692,
                                0.0431967899,
                                -0.0219510254,
                                -0.0118855191,
                                0.0125830323,
                                0.00954331178,
                                -0.00393587723,
                                0.0303479116,
                                -0.0311429743,
                                -0.0228111204,
                                0.0209214389,
                                0.101412728,
                                -0.0758953616,
                                -0.0196846407,
                                0.0439451411,
              

In [15]:
vector=index.query(id=list(),top_k=1,include_metadata=True)
print(vector)

{'matches': [{'id': 'What is YOLO in AI',
              'metadata': {'Answer': 'YOLO stands for "You Only Look Once" and '
                                     'it is a real-time object detection '
                                     'system in computer vision. YOLO is an '
                                     'algorithm that can detect and recognize '
                                     'multiple objects in an image or video '
                                     'frame, and it is known for its speed and '
                                     'accuracy. YOLO divides the input image '
                                     'into a grid and then predicts bounding '
                                     'boxes and class probabilities for each '
                                     'grid cell. This allows YOLO to '
                                     'efficiently detect and classify objects '
                                     'in real-time applications such as '
                       

In [36]:
import numpy as np
def get_ids_from_query(index,input_vector):
  print("searching pinecone...")
  results = index.query(vector=input_vector, top_k=10000,include_values=False)
  ids = set()
  print(type(results))
  for result in results['matches']:
    ids.add(result['id'])
  return ids

def get_all_ids_from_index(index, num_dimensions, namespace=""):
  num_vectors = index.describe_index_stats()["namespaces"][namespace]['vector_count']
  all_ids = set()
  while len(all_ids) < num_vectors:
    print("Length of ids list is shorter than the number of total vectors...")
    input_vector = np.random.rand(num_dimensions).tolist()
    print("creating random vector...")
    ids = get_ids_from_query(index,input_vector)
    print("getting ids from a vector query...")
    all_ids.update(ids)
    print("updating ids set...")
    print(f"Collected {len(all_ids)} ids out of {num_vectors}.")

  return all_ids

all_ids = list(get_all_ids_from_index(index, num_dimensions=768, namespace=""))
print(all_ids)


Length of ids list is shorter than the number of total vectors...
creating random vector...
searching pinecone...
<class 'pinecone.core.client.model.query_response.QueryResponse'>
getting ids from a vector query...
updating ids set...
Collected 4 ids out of 4.
['Tell me about AI', 'Describe YOLO', 'Who is Virat kohili', 'What is YOLO in AI']


In [39]:
diction={}
for i in all_ids:
    print(i)
    vectors=index.query(id=i,top_k=1,include_metadata=True)
    for vector in vectors['matches']:
        id,ans,counter=vector['id'],list(vector['metadata'].values())[0],list(vector['metadata'].values())[2]
        diction[id]=counter
print(diction)


Tell me about AI
Describe YOLO
Who is Virat kohili
What is YOLO in AI
{'Tell me about AI': 2.0, 'Describe YOLO': 0.0, 'Who is Virat kohili': 0.0, 'What is YOLO in AI': 1.0}


In [48]:
#Make the dictionary descending order of the counter
sorted_dict = sorted(diction.items(), key=lambda item: item[1], reverse=True)
print(sorted_dict)
new_dict = {key: value for key, value in sorted_dict}
# Printing the sorted dictionary
for id, count in new_dict.items():
    print(id, ' is asked ', count,' times')

[('Tell me about AI', 2.0), ('What is YOLO in AI', 1.0), ('Describe YOLO', 0.0), ('Who is Virat kohili', 0.0)]
Tell me about AI  is asked  2.0  times
What is YOLO in AI  is asked  1.0  times
Describe YOLO  is asked  0.0  times
Who is Virat kohili  is asked  0.0  times


In [6]:
id = []
for ids in index.list(prefix="", namespace="ns1"):
    # print(ids)
    # results1.append( index.fetch(ids=ids))
    # results1 = index.fetch(ids=ids)
    # print(results1)
    # print(ids)
    id = ids
 
print(id)

['Tell me about Prime Minster of India']


In [1]:
import pandas as pd
import numpy as np
from langchain.chat_models import AzureChatOpenAI
from chat2plot import chat2plot
numerical_data = {
    'x': np.arange(0, 10),
    'y': np.random.randint(0, 100, size=10)
}

plot = chat2plot(pd.DataFrame(numerical_data), chat=AzureChatOpenAI(api_key="836fea48dd7a402dbd8cf73f4dddb2e5",api_version="2023-05-15",model="BayerBIGPT35Turbo",azure_endpoint="https://bayerbichatgpt4.openai.azure.com/"))
ret = plot.query("plot a bar chart of x vs y")
ret.figure.show()
print(ret.explanation)


c:\Users\saediga\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
c:\Users\saediga\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Based on the user's request, we should plot a bar chart with the x-axis representing the 'x' column and the y-axis representing the 'y' column from the dataset. No filters need to be applied to the data. We don't need to transform the axes in any way. The chart type should be a bar chart.


In [3]:
pip freeze | findstr openai

openai==1.25.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
# List of numbers
list1 = [10, 20, 4, 45, 99]
 
# Printing the maximum element
print("Largest element is:", max(list1))

Largest element is: 99


In [7]:
n=30
amount=1
while n>0:
    amount=amount*2
    n=n-1
print(amount)

1073741824


In [8]:
n=30
amount=1

for i in range(1,n+1):
    amount=amount*2

print(amount)

1073741824


In [4]:
import re

def format_question(question):
    # Extract all numbers from the question
    matches = re.finditer(r'\d+', question)
    for match in matches:
        number = match.group(0)
        # Check if the number is less than 10 digits and greater than or equal to 8 digits
        if len(number) >= 8 and len(number) < 10:
            # Pad the number with leading zeroes to make it 10 digits
            formatted_number = number.zfill(10)
            # Replace the original number with the formatted number in the question
            question = question.replace(number, formatted_number, 1)
    return question

# Example usage
question = "what is the status 3052024 of my delivery 12345678900?"
modified_question = format_question(question)
print(modified_question)

what is the status 3052024 of my delivery 12345678900?
